# Interactive Data Profiling Notebook

This notebook demonstrates the automated data profiling system with interactive examples.

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import yaml

# Add src to path
sys.path.insert(0, str(Path.cwd() / 'src'))

from profiler.data_profiler import DataProfiler
from metadata.extractor import MetadataExtractor
from glossary.term_mapper import TermMapper
from dictionary.generator import DictionaryGenerator

## Step 1: Load Configuration and Data

In [ ]:
# Load configuration
with open('config/profiling_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Load sample data
df = pd.read_csv('data/sample_sales_data.csv')

print(f"Loaded dataset with {len(df)} rows and {len(df.columns)} columns")
df.head()

## Step 2: Profile the Dataset

In [ ]:
# Initialize profiler
profiler = DataProfiler(config)

# Profile the dataset
profile = profiler.profile_dataset(df, 'sample_sales_data')

# Display summary
print(profiler.get_summary('sample_sales_data'))

### View Column Profiles

In [ ]:
# Convert column profiles to DataFrame for easy viewing
column_stats = []
for col, stats in profile['column_profiles'].items():
    column_stats.append({
        'Column': col,
        'Type': stats['data_type'],
        'Nulls': f"{stats['null_percentage']:.1f}%",
        'Unique': stats['unique_count'],
        'Samples': ', '.join(str(s) for s in stats.get('sample_values', [])[:3])
    })

pd.DataFrame(column_stats)

### Data Quality Assessment

In [ ]:
quality = profile['data_quality']

print(f"Overall Completeness: {quality['overall_completeness']:.2f}%")
print(f"Duplicate Rows: {quality['duplicate_rows_count']}")
print(f"Quality Issues: {len(quality['quality_issues'])}")

if quality['quality_issues']:
    print("\nQuality Issues:")
    for issue in quality['quality_issues']:
        print(f"  - {issue['column']}: {issue['issue']}")

## Step 3: Extract Metadata

In [ ]:
# Initialize metadata extractor
metadata_extractor = MetadataExtractor(config)

# Extract metadata
source_info = {'system': 'CSV', 'table': 'sample_sales_data'}
metadata = metadata_extractor.extract_metadata(df, 'sample_sales_data', source_info)

# Display schema
schema_df = pd.DataFrame(metadata['schema'])
schema_df[['column_name', 'data_type', 'sql_type', 'nullable']]

### Generate DDL Statement

In [ ]:
ddl = metadata_extractor.generate_schema_ddl('sample_sales_data', 'sales_table', 'postgresql')
print(ddl)

## Step 4: Map to Business Glossary

In [ ]:
# Initialize term mapper
term_mapper = TermMapper('config/business_terms.yaml', config)

# Map columns
mappings = term_mapper.map_columns(list(df.columns), 'sample_sales_data')

# Display mappings
mapping_data = []
for col, mapping in mappings.items():
    mapping_data.append({
        'Technical': col,
        'Business': mapping['business_name'],
        'Mapped': '✓' if mapping['mapped'] else '✗',
        'PII': '🔒' if mapping.get('is_pii') else ''
    })

pd.DataFrame(mapping_data)

### Identify PII Columns

In [ ]:
pii_columns = term_mapper.identify_pii_columns('sample_sales_data')
print(f"PII Columns: {', '.join(pii_columns) if pii_columns else 'None detected'}")

## Step 5: Generate Data Dictionary

In [ ]:
# Initialize dictionary generator
dict_generator = DictionaryGenerator(config)

# Get glossary
glossary = term_mapper.get_business_glossary('sample_sales_data')

# Generate dictionary
dictionary = dict_generator.generate_dictionary(
    'sample_sales_data', profile, metadata, glossary
)

# Display overview
print("Data Dictionary Overview:")
print(f"Dataset: {dictionary['dataset_name']}")
print(f"Records: {dictionary['overview']['record_count']:,}")
print(f"Fields: {dictionary['overview']['field_count']}")
print(f"Quality Score: {dictionary['data_quality']['quality_score']}")

### View Sample Column Definition

In [ ]:
# Display detailed info for first column
sample_col = dictionary['columns'][0]
print(f"Business Name: {sample_col['business_name']}")
print(f"Technical Name: {sample_col['technical_name']}")
print(f"Description: {sample_col['description']}")
print(f"Data Type: {sample_col['data_type']['sql']}")
print(f"Null Rate: {sample_col['statistics']['null_percentage']}")
if sample_col.get('sample_values'):
    print(f"Samples: {', '.join(sample_col['sample_values'])}")

## Bonus: Custom Analysis

In [ ]:
# Analyze correlations for numeric columns
if profile.get('correlations'):
    print("Strong Correlations Found:")
    for corr in profile['correlations'].get('strong_correlations', []):
        print(f"  {corr['column_1']} <-> {corr['column_2']}: {corr['correlation']:.2f}")

In [ ]:
# Pattern detection results
if profile.get('patterns'):
    print("Patterns Detected:")
    for col, patterns in profile['patterns'].items():
        print(f"\n{col}:")
        for pattern_type, match_info in patterns.items():
            print(f"  - {pattern_type}: {match_info}")

## Export Results

In [ ]:
from pathlib import Path

# Create output directories
outputs_path = Path('outputs')
outputs_path.mkdir(exist_ok=True)

# Export dictionary as HTML
dict_generator.export_html('sample_sales_data', outputs_path)

# Export dictionary as Markdown
dict_generator.export_markdown('sample_sales_data', outputs_path)

# Export glossary as CSV
term_mapper.export_glossary('sample_sales_data', outputs_path, format='csv')

print("\nExports complete! Check the outputs/ directory.")